In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from tensorflow.keras.datasets import mnist

In [9]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [10]:
x_train.shape

(60000, 28, 28)

## 一個channel，所以(28,28)-->(28,28,1)

In [11]:
x_train = x_train.reshape(60000, 28, 28, 1) /255

In [13]:
x_test = x_test.reshape(10000, 28, 28, 1) /255

In [15]:
from tensorflow.keras.utils import to_categorical

In [16]:
y_train = to_categorical(y_train, 10)

In [17]:
y_test = to_categorical(y_test, 10)

## 讀入必要的函式

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

### Step 1 打造函數學習機(CNN)

In [21]:
model = Sequential()

In [23]:
model.add(Conv2D(16, (3,3), padding='same', #第一層
                input_shape=(28,28,1),
                activation='relu')) #輸出16個28x28矩陣，事實上是(28, 28, 16)

In [24]:
model.add(MaxPooling2D(pool_size=(2,2))) #output (14, 14, 16)

In [25]:
model.add(Conv2D(32, (3,3), padding='same', #第二層
                activation='relu')) #output (14, 14, 32)

In [27]:
model.add(MaxPooling2D(pool_size=(2,2))) #output (7, 7, 32)

In [28]:
model.add(Conv2D(64, (3,3), padding='same', #第三層
                activation='relu')) #output (7, 7, 64)

In [29]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [30]:
model.add(Flatten())

In [31]:
model.add(Dense(54, activation='relu'))

In [32]:
model.add(Dense(10, activation='softmax')) #讓最後數字加起來等於1

## 看一下神經網路

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0

In [34]:
# 3*3 (權重) + 1 (bias)

In [35]:
model.compile(loss='mse', optimizer=SGD(lr=0.087), #可以試試不同的loss function
             metrics=['accuracy'])

### Step 2 fit

In [36]:
model.fit(x_train, y_train, batch_size=128, epochs=12)

Epoch 1/12
60000/60000 [==============================] - 40s 666us/sample - loss: 0.0896 - acc: 0.1822
Epoch 2/12
60000/60000 [==============================] - 39s 644us/sample - loss: 0.0883 - acc: 0.3856
Epoch 3/12
60000/60000 [==============================] - 38s 637us/sample - loss: 0.0803 - acc: 0.4804
Epoch 4/12
60000/60000 [==============================] - 39s 648us/sample - loss: 0.0372 - acc: 0.7695
Epoch 5/12
60000/60000 [==============================] - 42s 705us/sample - loss: 0.0173 - acc: 0.8910
Epoch 6/12
60000/60000 [==============================] - 39s 657us/sample - loss: 0.0126 - acc: 0.9206
Epoch 7/12
60000/60000 [==============================] - 42s 695us/sample - loss: 0.0101 - acc: 0.9369
Epoch 8/12
60000/60000 [==============================] - 41s 676us/sample - loss: 0.0087 - acc: 0.9449 - lo
Epoch 9/12
60000/60000 [==============================] - 41s 683us/sample - loss: 0.0077 - acc: 0.9513
Epoch 10/12
60000/60000 [==============================] - 

### Step 3 預測

In [37]:
result = model.predict_classes(x_test)

In [38]:
def my_predict(n):
    print('我可愛的CNN預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [42]:
from ipywidgets import interact_manual

In [45]:
interact_manual(my_predict, n=(0, 9999))

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

<function __main__.my_predict(n)>

In [46]:
score = model.evaluate(x_test, y_test)

10000/10000 [==============================] - 2s 234us/sample - loss: 0.0061 - acc: 0.9601


In [47]:
loss, acc = score

In [48]:
print('測試資料正確率', acc)

測試資料正確率 0.9601


## 把我們的model存起來

In [49]:
model.save('myCNNmodel.h5')